<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%206/Introducci%C3%B3n_a_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gensim

## Tokenización

In [ ]:
from gensim.utils import tokenize
from gensim.utils import simple_preprocess

In [ ]:
texto = 'Este es un texto de prueba. ¡Tiene puntuación, acentos y números 1, 2, 3! Este _token es algo raro'

Con ```tokenize``` obtenemos el texto sin números ni signos de puntuación

In [ ]:
list(tokenize(texto, to_lower=True))

['este',
 'es',
 'un',
 'texto',
 'de',
 'prueba',
 'tiene',
 'puntuación',
 'acentos',
 'y',
 'números',
 'este',
 '_token',
 'es',
 'algo',
 'raro']

```simple_preprocess``` agrega unas cuentas reglas a lo que se considera un token. Específicamente, no puede empezar con '_' y debe tener entre 2 y 15 letras. 

In [ ]:
simple_preprocess(texto)

['este',
 'es',
 'un',
 'texto',
 'de',
 'prueba',
 'tiene',
 'puntuación',
 'acentos',
 'números',
 'este',
 'es',
 'algo',
 'raro']

El parámetro ```deacc = True``` elimina acentos.

In [ ]:
simple_preprocess(texto, deacc = True)

['este',
 'es',
 'un',
 'texto',
 'de',
 'prueba',
 'tiene',
 'puntuacion',
 'acentos',
 'numeros']

Podemos tokenizar varios documentos con list comprehension. 

In [ ]:
docs = [
        '¡Primer documento!', 
        "Este es el segundo documento.", 
        'Este es el tercer documento, con un poco más de texto que los demás.'
]

In [ ]:
tokenized_docs = [simple_preprocess(doc, deacc = True) for doc in docs]
tokenized_docs

[['primer', 'documento'],
 ['este', 'es', 'el', 'segundo', 'documento'],
 ['este',
  'es',
  'el',
  'tercer',
  'documento',
  'con',
  'un',
  'poco',
  'mas',
  'de',
  'texto',
  'que',
  'los',
  'demas']]

¿Cómo eliminar stopwords con estas funciones?

## Dictionary

gensim tiene una clase llamada ```Dictionary``` cuya utilidad principal es la de asignar un id a cada palabra y crear el corpus para entrenar los modelos.

In [ ]:
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary([simple_preprocess(doc, deacc = True) for doc in docs])
print(dictionary)

Dictionary(16 unique tokens: ['documento', 'primer', 'el', 'es', 'este']...)


In [ ]:
dictionary.token2id

{'con': 6,
 'de': 7,
 'demas': 8,
 'documento': 0,
 'el': 2,
 'es': 3,
 'este': 4,
 'los': 9,
 'mas': 10,
 'poco': 11,
 'primer': 1,
 'que': 12,
 'segundo': 5,
 'tercer': 13,
 'texto': 14,
 'un': 15}

Podemos agregar documentos a nuestro a diccionario ocupando la función ```add_documents```.

In [ ]:
docs_2 = [
          'Este es otro documento que no estaba', 
          'Aparecerá la palabra taquito por este documento'
]

dictionary.add_documents([simple_preprocess(doc, deacc = True) for doc in docs_2])
print(dictionary)

Dictionary(25 unique tokens: ['documento', 'primer', 'el', 'es', 'este']...)


In [ ]:
dictionary.token2id

{'aparecera': 24,
 'aparecerá': 19,
 'con': 6,
 'de': 7,
 'demas': 8,
 'documento': 0,
 'el': 2,
 'es': 3,
 'estaba': 16,
 'este': 4,
 'la': 20,
 'los': 9,
 'mas': 10,
 'no': 17,
 'otro': 18,
 'palabra': 21,
 'poco': 11,
 'por': 22,
 'primer': 1,
 'que': 12,
 'segundo': 5,
 'taquito': 23,
 'tercer': 13,
 'texto': 14,
 'un': 15}

## Bolsa de Palabras

Para crear la bolsa de palabras basta que usamos la función ```doc2bow```. 

La función espera un documento tokenizado y regresa una lista con tuplas (word_id, word_count).

Es muy parecido a ```skelarn.feature_extraction.text.CountVectorizer```. Crear el diccionario es lo equivalente a usar el método ```fit``` y ```doc2bow``` es equivalente a ```transform```.

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in [simple_preprocess(doc) for doc in docs + docs_2]]
corpus

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (6, 1),
  (7, 1),
  (9, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1)],
 [(0, 1), (3, 1), (4, 1), (12, 1), (16, 1), (17, 1), (18, 1)],
 [(0, 1), (4, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]]

Para hacerlo claro para nosotros, podemos imprimir la palabra en lugar del id de la siguiente manera

In [ ]:
[[(id, dictionary[id], count) for id, count in doc] for doc in corpus]

[[(0, 'documento', 1), (1, 'primer', 1)],
 [(0, 'documento', 1),
  (2, 'el', 1),
  (3, 'es', 1),
  (4, 'este', 1),
  (5, 'segundo', 1)],
 [(0, 'documento', 1),
  (2, 'el', 1),
  (3, 'es', 1),
  (4, 'este', 1),
  (6, 'con', 1),
  (7, 'de', 1),
  (9, 'los', 1),
  (11, 'poco', 1),
  (12, 'que', 1),
  (13, 'tercer', 1),
  (14, 'texto', 1),
  (15, 'un', 1)],
 [(0, 'documento', 1),
  (3, 'es', 1),
  (4, 'este', 1),
  (12, 'que', 1),
  (16, 'estaba', 1),
  (17, 'no', 1),
  (18, 'otro', 1)],
 [(0, 'documento', 1),
  (4, 'este', 1),
  (19, 'aparecerá', 1),
  (20, 'la', 1),
  (21, 'palabra', 1),
  (22, 'por', 1),
  (23, 'taquito', 1)]]

In [ ]:
docs + docs_2

['¡Primer documento!',
 'Este es el segundo documento.',
 'Este es el tercer documento, con un poco más de texto que los demaś.',
 'Este es otro documento que no estaba',
 'Aparecerá la palabra taquito por este documento']

In [ ]:
dictionary.doc2bow(simple_preprocess('Es mejor comer un taquito al día que ningún taquito'))

[(3, 1), (12, 1), (15, 1), (23, 2)]

## Salvar el diccionario y el corpus

In [ ]:
dictionary.save('dictionary.dict')
corpora.MmCorpus.serialize('bow_corpus.mm', corpus)

In [ ]:
loaded_dict = corpora.Dictionary.load('dictionary.dict')
loaded_corpus = corpora.MmCorpus('bow_corpus.mm')

In [ ]:
list(loaded_corpus)

[[(0, 1.0), (1, 1.0)],
 [(0, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)],
 [(0, 1.0),
  (2, 1.0),
  (3, 1.0),
  (4, 1.0),
  (6, 1.0),
  (7, 1.0),
  (9, 1.0),
  (11, 1.0),
  (12, 1.0),
  (13, 1.0),
  (14, 1.0),
  (15, 1.0)],
 [(0, 1.0), (3, 1.0), (4, 1.0), (12, 1.0), (16, 1.0), (17, 1.0), (18, 1.0)],
 [(0, 1.0), (4, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0), (23, 1.0)]]

## TfIdf

Para crear la BoW con tfidf usamos ```gensim.models.TfidfModel```. Haciendo la comparación con sklearn, es como si primero ocuparamos un ```CountVectorizer``` y luego un ```TfidfTransformer```.

In [ ]:
from gensim import models

In [ ]:
tfidf = models.TfidfModel(corpus = loaded_corpus)
tfidf

In [ ]:
tfidf[corpus[0]]

[(1, 1.0)]

In [ ]:
tfidf[corpus[1]]

[(2, 0.4737637094387418),
 (3, 0.26411992828600717),
 (4, 0.11537529839652863),
 (5, 0.8321521204809549)]

El modelo de TfIdf ya no consideró la palabra con id 0 (documento).

## word2vec

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec

### Modelo entrenado

In [ ]:
# api.info()
api.info('glove-wiki-gigaword-50')

{'base_dataset': 'Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)',
 'checksum': 'c289bc5d7f2f02c6dc9f2f9b67641813',
 'description': 'Pre-trained vectors based on Wikipedia 2014 + Gigaword, 5.6B tokens, 400K vocab, uncased (https://nlp.stanford.edu/projects/glove/).',
 'file_name': 'glove-wiki-gigaword-50.gz',
 'file_size': 69182535,
 'license': 'http://opendatacommons.org/licenses/pddl/',
 'num_records': 400000,
 'parameters': {'dimension': 50},
 'parts': 1,
 'preprocessing': 'Converted to w2v format with `python -m gensim.scripts.glove2word2vec -i <fname> -o glove-wiki-gigaword-50.txt`.',
 'read_more': ['https://nlp.stanford.edu/projects/glove/',
  'https://nlp.stanford.edu/pubs/glove.pdf'],
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/glove-wiki-gigaword-50/__init__.py'}

In [ ]:
model = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


#### Palabras similares

In [ ]:
model.most_similar('mexico')

[('mexican', 0.8550674319267273),
 ('venezuela', 0.8496899008750916),
 ('colombia', 0.8490317463874817),
 ('peru', 0.8446482419967651),
 ('chile', 0.8439290523529053),
 ('puerto', 0.8362628221511841),
 ('rico', 0.8194695711135864),
 ('cuba', 0.8125205636024475),
 ('guatemala', 0.8113811016082764),
 ('panama', 0.8096755743026733)]

In [ ]:
print(model.similarity('mexico', 'mariachi'))
print(model.similarity('usa', 'mariachi'))
print(model.similarity('alemania', 'mariachi'))

0.3073805
0.07876266
-0.014929158


#### Encontrar el valor extraño en un conjunto de valores

In [ ]:
model.doesnt_match(['guitar', 'violin', 'piano', 'tablet'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'tablet'

In [ ]:
model.doesnt_match(['guitar', 'violin', 'piano', 'drums'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'drums'

#### Analogías

In [ ]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.8523603677749634),
 ('throne', 0.7664334177970886),
 ('prince', 0.759214460849762),
 ('daughter', 0.7473883032798767),
 ('elizabeth', 0.7460220456123352),
 ('princess', 0.7424569725990295),
 ('kingdom', 0.7337411642074585),
 ('monarch', 0.7214490175247192),
 ('eldest', 0.7184861898422241),
 ('widow', 0.7099430561065674)]

In [ ]:
model.most_similar(positive=['car', 'air'], negative=['road'])

[('jet', 0.7882938385009766),
 ('airplane', 0.7694613933563232),
 ('aircraft', 0.7238950133323669),
 ('planes', 0.7226904630661011),
 ('plane', 0.7093905806541443),
 ('pilot', 0.707859992980957),
 ('airplanes', 0.702531099319458),
 ('pilots', 0.6944992542266846),
 ('c-130', 0.67693030834198),
 ('fighter', 0.6713298559188843)]

In [ ]:
model.most_similar(positive=['lawyer', 'medicine'], negative=['justice'])

[('physician', 0.7534809112548828),
 ('medical', 0.7412653565406799),
 ('dentist', 0.7200738191604614),
 ('pediatrician', 0.7030641436576843),
 ('nursing', 0.6904869675636292),
 ('cardiologist', 0.6887562274932861),
 ('pharmacist', 0.6869213581085205),
 ('internist', 0.6854752898216248),
 ('clinic', 0.6843182444572449),
 ('veterinary', 0.6726669669151306)]

### Entrenar un modelo propio

In [ ]:
import zipfile
with zipfile.ZipFile('/content/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/friends_dataset/')

In [ ]:
#https://datascience.stackexchange.com/questions/11077/using-several-documents-with-word2vec
import os

class WordTrainer(object):
   def __init__(self, dir_name):
      self.dir_name = dir_name
   def __iter__(self):
      for _,file_name in enumerate(os.listdir(self.dir_name)):
          for _,line in enumerate(open(os.path.join(self.dir_name, file_name),'r')):
              words = simple_preprocess(line)
              yield words

screen_play = WordTrainer('/content/friends_dataset')
my_model = Word2Vec(screen_play, size=300, window=15, min_count=5)

In [ ]:
my_model.wv.most_similar('ross')

[('joey', 0.7847803831100464),
 ('chandler', 0.7796059846878052),
 ('rachel', 0.7070483565330505),
 ('phoebe', 0.6731643676757812),
 ('monica', 0.6516903638839722),
 ('tag', 0.6110426187515259),
 ('richard', 0.6073410511016846),
 ('paul', 0.5925347805023193),
 ('pete', 0.5805870294570923),
 ('mike', 0.57646244764328)]

In [ ]:
my_model.wv.doesnt_match(['rachel', 'ross', 'phoebe', 'chandler', 'joey'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'joey'

In [ ]:
my_model.wv.most_similar(['ross', 'actor'], ['paleontologist'], topn=20)

[('but', 0.5867981910705566),
 ('because', 0.5396002531051636),
 ('that', 0.5296133756637573),
 ('big', 0.5232568979263306),
 ('cause', 0.5188854336738586),
 ('mean', 0.5145672559738159),
 ('thing', 0.5111375451087952),
 ('just', 0.5063825845718384),
 ('person', 0.5029212236404419),
 ('pause', 0.49938642978668213),
 ('important', 0.4991603195667267),
 ('chandler', 0.48641732335090637),
 ('joey', 0.481323778629303),
 ('though', 0.4758177697658539),
 ('more', 0.47416940331459045),
 ('stupid', 0.47282907366752625),
 ('than', 0.47001761198043823),
 ('reason', 0.46713393926620483),
 ('it', 0.4650190770626068),
 ('idea', 0.46199238300323486)]